# 1. How to init any model in one line

## Basic usage

In [1]:
from langchain.chat_models import init_chat_model

gemini = init_chat_model("gemini-2.5-flash", model_provider="google_genai", temperature=0)

print("Gemini: " + gemini.invoke("what's your name?").content)

Gemini: I do not have a name. I am a large language model, trained by Google.


##  Inferring model provider

## Creating a configurable model

In [3]:
configurable_model = init_chat_model(temperature=0)

configurable_model.invoke(
  "what's your name", config={"configurable": {"model": "gemini-2.5-flash", "model_provider": "google_genai"}}
)

AIMessage(content='I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--4087807a-4a3d-47e6-ba42-2af48ebfba90-0', usage_metadata={'input_tokens': 6, 'output_tokens': 150, 'total_tokens': 156, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 139}})

### Configurable model with default values

In [8]:
first_llm = init_chat_model(
  model="gemini-2.5-flash",
  model_provider="google_genai",
  temperature=0,
  configurable_fields=("model", "model_provider", "temperature", "max_tokens"),
  config_prefix="first"
)

first_llm.invoke("what's your name?")

AIMessage(content='I do not have a name. I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--45868077-2a0d-4ce5-b9d3-1695b9f724f9-0', usage_metadata={'input_tokens': 7, 'output_tokens': 53, 'total_tokens': 60, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 35}})

In [9]:
first_llm.invoke(
  "what's your name",
  config={
    "configurable": {
      "first_model": "gemini-2.5-flash-lite",
      "first_temperature": 0.8,
      "first_max_tokens:": 100
    }
  }
)

AIMessage(content='I do not have a name. I am a large language model, trained by Google.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash-lite', 'safety_ratings': []}, id='run--adccc4b4-f186-414f-8672-734d158d1e5f-0', usage_metadata={'input_tokens': 6, 'output_tokens': 18, 'total_tokens': 24, 'input_token_details': {'cache_read': 0}})

### Using a configurable model declaratively

In [10]:
from pydantic import BaseModel, Field

class GetWeather(BaseModel):
   """Get the current weather in a given location"""
   location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

class GetPopulation(BaseModel):
  """Get the current population in a given location"""
  location: str = Field(..., description="The city and state, e.g. San Francisco, CA")

llm = init_chat_model(temperature=0)
llm_with_tools = llm.bind_tools([GetWeather, GetPopulation])

llm_with_tools.invoke(
  "what's bigger in 2024 LA or NYC", config={"configurable": {"model": "gemini-2.5-flash", "model_provider": "google_genai"}}
).tool_calls

[{'name': 'GetPopulation',
  'args': {'location': 'Los Angeles, CA'},
  'id': 'd2ccb0a3-193b-45bf-9fea-40b26b41ad64',
  'type': 'tool_call'},
 {'name': 'GetPopulation',
  'args': {'location': 'New York, NY'},
  'id': '6298c11f-9450-4aa9-a175-98d022dad284',
  'type': 'tool_call'}]